## Scraping New Music Friday
This notebook scrapes the Spotify New Music Friday playlist. We are collecting this to serve as a "test" dataset for our predictions model. Even though we will not have the popularity outcomes for these songs, we can still use these tracks to serve as example outputs for our model.

*Note: This data was collected on 9/29/2023.*

#### Set-Up

In [8]:
# import libraries and functions
import os
from dotenv import load_dotenv
from functions.spotify_api import SpotifyAPI
import json

In [2]:
# load secrets
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# init api
spotify = SpotifyAPI(client_id=client_id, client_secret=client_secret)

#### Request Playlist

In [3]:
# get playlist
playlist_id = "37i9dQZF1DX4JAvHpjipBk"
playlist_res = spotify.request_playlist(playlist_id=playlist_id)

#### Process Playlist Tracks

In [4]:
playlist_name = playlist_res["name"]

# init list
tracks = []

print("-- processing playlist: " + playlist_name, "--")
# extract tracks
playlist_tracks = playlist_res["tracks"]["items"]
# loop through tracks
for track in playlist_tracks:
    # parse track
    track_json = spotify.parse_playlist_track(track=track, playlist_id=playlist_id, playlist_name=playlist_name)
    # append to list
    tracks.append(track_json)

-- processing playlist: New Music Friday --


#### Scrape Artists

*Note: Takes a while to run*

In [5]:
# get artist ids
artist_ids = list(set([artist["id"] for playlist in tracks for artist in playlist["artist_ids"]]))

# init list
artists = []

for artist_id in artist_ids:
    artists.append(spotify.request_artist(artist_id=artist_id, silent=True))

#### Scrape Audio Features

*Note: Takes a while to run.*

In [6]:
# get track ids
track_ids = list(set([track["track_id"] for track in tracks]))

# init list
audio_features = []

for track_id in track_ids:
    audio_features.append(spotify.request_audio_features(track_id=track_id, silent=True))

#### Write Data Outputs

In [10]:
# parent playlist
with open("../../data/test_data/nmf_playlist.json", "w") as f:
    json.dump(tracks, f)

# artists
with open("../../data/test_data/nmf_artists.json", "w") as f:
    json.dump(artists, f)

# audio features
with open("../../data/test_data/nmf_audio_features.json", "w") as f:
    json.dump(audio_features, f)